In [5]:
import sys
import os

# Agregamos el directorio actual al path
sys.path.append(os.path.abspath("."))

# Intentamos el import
try:
    from minitorch.tensor import Tensor
    print("✅ Importación directa desde archivo exitosa.")
    
    import minitorch
    from minitorch import Tensor
    print("✅ Importación desde paquete (__init__) exitosa.")
except ImportError as e:
    print(f"❌ Error: {e}")
    # Esto te dirá dónde está buscando Python realmente
    print(f"Buscando en: {sys.path[:3]}")

❌ Error: No module named 'minitorch.tensor'
Buscando en: ['C:\\Users\\SoFanty\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\SoFanty\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\SoFanty\\AppData\\Local\\Programs\\Python\\Python313\\Lib']
